In [1]:
from utils import *

In [2]:
# load data

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("tmp/")

Extracting tmp/train-images-idx3-ubyte.gz
Extracting tmp/train-labels-idx1-ubyte.gz
Extracting tmp/t10k-images-idx3-ubyte.gz
Extracting tmp/t10k-labels-idx1-ubyte.gz


If you have access to the original graph construction code, you can use that to create the graph with modifications to the new layers, then load weight *from certain layers* into the graph. The rest will be initialized per your specifications.

In [4]:
# construction

reset_graph()

# We dropped layer 5 and reduced layer 4 to 20 neurons.
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 20
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4")
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.01
threshold = 1.0
    
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    grads_and_vars = optimizer.compute_gradients(loss)
    capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var) for grad, var in grads_and_vars]
    training_op = optimizer.apply_gradients(capped_gvs)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [7]:
# get the weights to load. the trick here is the regular expression in scope: it'll only 
# get the weights for hidden 1 2 and 3
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                              scope="hidden[123]")
# put the vars to pull into the proper format for saver
# dictionary with keys as name and values as the variables themselves
reuse_vars_dict = dict([(var.op.name, var) for var in reuse_vars])

# this saver will pull the weights we need
# argument are vars dict of vars to restore
restore_saver = tf.train.Saver(reuse_vars_dict)

init = tf.global_variables_initializer()
# we need a new saver if we want to save this run of the model
saver = tf.train.Saver()

In [9]:
# execution
n_epochs = 20
batch_size = 200

with tf.Session() as sess:
    init.run()
    # restoring only the weights we asked for, hidden123
    restore_saver.restore(sess, "savedmodels/11_07_gradientclipping.ckpt")
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        if epoch % 5 == 0 or epoch == n_epochs - 1:
            acc_train = accuracy.eval(feed_dict={X:mnist.train.images, y:mnist.train.labels})
            acc_val = accuracy.eval(feed_dict={X:mnist.validation.images, y:mnist.validation.labels})
            print(epoch, "train acc:", acc_train, "val acc:", acc_val)
    acc_test = accuracy.eval(feed_dict={X:mnist.test.images, y:mnist.test.labels})
    print("Test acc:", acc_test)

INFO:tensorflow:Restoring parameters from savedmodels/11_07_gradientclipping.ckpt
0 train acc: 0.898636 val acc: 0.9028
5 train acc: 0.953491 val acc: 0.9546
10 train acc: 0.963364 val acc: 0.9614
15 train acc: 0.970273 val acc: 0.9662
19 train acc: 0.973545 val acc: 0.9686
Test acc: 0.9653
